In [ ]:
# import modules
import pandas as pd
import numpy as np


In [ ]:
# scrape data and check for number of records per year
years = (2015, 2016, 2017, 2018, 2019)

all_COLI_data = []

for year in years:
    url = 'https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=' + str(year)
    print(url)
    COLI_data = pd.read_html(url)
    print(len(COLI_data[2]))
    all_COLI_data.append(COLI_data[2])


In [ ]:
# create a dataframe for each year
COLI_df_2015 = pd.DataFrame(all_COLI_data[0])
COLI_df_2016 = pd.DataFrame(all_COLI_data[1])
COLI_df_2017 = pd.DataFrame(all_COLI_data[2])
COLI_df_2018 = pd.DataFrame(all_COLI_data[3])
COLI_df_2019 = pd.DataFrame(all_COLI_data[4])

# add a column for year and assigning year value
COLI_df_2015["Year"] = [int(2015)]*len(all_COLI_data[0]) 
COLI_df_2016["Year"] = [int(2016)]*len(all_COLI_data[1]) 
COLI_df_2017["Year"] = [int(2017)]*len(all_COLI_data[2]) 
COLI_df_2018["Year"] = [int(2018)]*len(all_COLI_data[3]) 
COLI_df_2019["Year"] = [int(2019)]*len(all_COLI_data[4]) 

# add annual ranking based on row number + 1
COLI_df_2015["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2015)+1, step = 1)
COLI_df_2016["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2016)+1, step = 1)
COLI_df_2017["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2017)+1, step = 1)
COLI_df_2018["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2018)+1, step = 1)
COLI_df_2019["Ranking"] = np.arange(start = 1, stop = len(COLI_df_2019)+1, step = 1)


In [ ]:
# concatenate dataframes for all years to find common countries across all years
result_df = pd.concat([COLI_df_2015,COLI_df_2016, COLI_df_2017, COLI_df_2018, COLI_df_2019],
                      axis=1, 
                      join="inner"
                     )
result_df

In [ ]:
# slice dataframes to create dataframes for each year
COLI_df_2015 = result_df.iloc[:,1:10]
COLI_df_2016 = result_df.iloc[:,11:20]
COLI_df_2017 = result_df.iloc[:,21:30]
COLI_df_2018 = result_df.iloc[:,31:40]
COLI_df_2019 = result_df.iloc[:,41:50]


In [ ]:
# concatenate dataframes for all years to find common countries across all years
COLI_df = pd.concat([COLI_df_2015,COLI_df_2016, COLI_df_2017, COLI_df_2018, COLI_df_2019
                    ], 
                    axis=0, 
                    join="inner"
                   )

In [ ]:
COLI_df

In [ ]:
# rename columns to remove spacing
COLI_df.rename(columns={"Cost of Living Index":"CostOfLiving_index",
                        "Rent Index":"Rent_index",
                        "Groceries Index":"Groceries_index",
                        "Restaurant Price Index":"RestaurantPrice_index",
                        "Local Purchasing Power Index":"LocalPurchPower_index"
                       }, 
               inplace=True
              )

In [ ]:
# slice dataframe (keeping relevant columns) and rearrange column order
COLI_df = COLI_df[["Country", "Year", "Ranking", "CostOfLiving_index", "Rent_index", "Groceries_index", 
                   "RestaurantPrice_index","LocalPurchPower_index"
                  ]
                 ] 

In [ ]:
# display dataframe
COLI_df


In [ ]:
# Scrape data and check for number of records per year
all_property_data = []

for year in years:
    url = 'https://www.numbeo.com/property-investment/rankings_by_country.jsp?title=' + str(year)
    print(url)
    property_data = pd.read_html(url)
    print(len(property_data[2]))
    all_property_data.append(property_data[2])


In [ ]:
# create a dataframe for each year
property_df_2015 = pd.DataFrame(all_property_data[0])
property_df_2016 = pd.DataFrame(all_property_data[1])
property_df_2017 = pd.DataFrame(all_property_data[2])
property_df_2018 = pd.DataFrame(all_property_data[3])
property_df_2019 = pd.DataFrame(all_property_data[4])

# add a column for year and assigning year value
property_df_2015["Year"] = [int(2015)]*len(all_property_data[0]) 
property_df_2016["Year"] = [int(2016)]*len(all_property_data[1]) 
property_df_2017["Year"] = [int(2017)]*len(all_property_data[2]) 
property_df_2018["Year"] = [int(2018)]*len(all_property_data[3]) 
property_df_2019["Year"] = [int(2019)]*len(all_property_data[4]) 


In [ ]:
# concatenate dataframes for all years to find common countries across all years
property_result_df = pd.concat([property_df_2015,property_df_2016, property_df_2017, 
                                property_df_2018, property_df_2019
                               ], 
                               axis=1, 
                               join="inner"
                              )
property_result_df

In [ ]:
# slice dataframes to create dataframes for each year
property_df_2015 = property_result_df.iloc[:,1:10]
property_df_2016 = property_result_df.iloc[:,11:20]
property_df_2017 = property_result_df.iloc[:,21:30]
property_df_2018 = property_result_df.iloc[:,31:40]
property_df_2019 = property_result_df.iloc[:,41:50]


In [ ]:
# concatenate dataframes for all years to find common countries across all years
property_df = pd.concat([property_df_2015, property_df_2016, property_df_2017,
                         property_df_2018, property_df_2019
                        ], 
                        axis=0, 
                        join="inner"
                       )

In [ ]:
# slice dataframe (keeping relevant columns) and rearrange column order
property_df = property_df[["Country", "Year", "Price To Income Ratio", "Mortgage As A Percentage Of Income", 
                           "Affordability Index"
                          ]
                         ]
property_df

In [ ]:
 # rename columns to remove spacing
property_df.rename(columns={"Price To Income Ratio":"price_income_ratio",
                            "Mortgage As A Percentage Of Income":"mortage_income_ratio",
                            "Affordability Index":"affordability_index"
                           }, 
                   inplace=True
                  )

In [ ]:
property_df

In [ ]:
#COLI_df.to_csv("COLI_raw.csv")
#property_df.to_csv("property_raw.csv")